# **ANÁLISIS EXPLORATORIO DE LOS DATOS**

In [15]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imblearn
from functools import reduce
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore", category=UserWarning)

In [16]:
df_final = pd.read_csv('df_final.csv')
df_final

,Unnamed: 0,ehr,n_tumor,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,...,er,her2,ki67,pr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age
0,0,10011773,1,IS,0,0,,,,0,...,0.0,0.0,19.0,1.0,0.0,0.0,0.0,0.0,13.000000,49.0
1,1,10020495,1,1,0,0,,,,IA,...,1.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0
2,2,10030299,1,1,0,0,,,,IA,...,1.0,0.0,18.0,1.0,3.0,3.0,0.0,0.0,13.000000,49.0
3,3,10030824,1,2,1,0,2,2.0,0.0,IIIA,...,1.0,0.0,18.0,1.0,0.0,0.0,0.0,0.0,13.000000,44.0
4,4,10041592,1,1,1,0,4,0.0,0.0,IB,...,0.0,0.0,65.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9966,9966,99966287,1,1,1,0,1,0.0,0.0,IB,...,1.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0
9967,9967,99981192,1,1,0,0,,,,IA,...,1.0,0.0,14.0,0.0,3.0,3.0,0.0,0.0,11.000000,49.0
9968,9968,99988958,1,0,2,0,1,1.0,0.0,IIIA,...,1.0,0.0,67.0,1.0,0.0,0.0,0.0,0.0,13.110481,49.0
9969,9969,99992350,1,2,0,0,3,2.0,0.0,IIA,...,0.0,0.0,16.0,0.0,4.0,3.0,1.0,0.0,11.000000,55.0


Vamos a ver cuántos pacientes han tenido una recaída, tras hablar con los expertos del dominio (en este caso nosotros) , han considerado que tener una recaída del tumor signfica haber tenido 2 tumores, por ello, hemos obtenido aquellos pacientes que si que presentan 2 tumores...

In [17]:
df_final['recaida'] = (df_final['n_tumor'] != df_final['n_tumor_2']).astype(int)
df_final[df_final['recaida'] == 1]


,Unnamed: 0,ehr,n_tumor,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,...,her2,ki67,pr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age,recaida
1,1,10020495,1,1,0,0,,,,IA,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0,1
17,17,10144916,1,1,0,0,1,0.0,1.0,IA,...,0.0,20.0,1.0,0.0,0.0,0.0,0.0,14.000000,49.0,1
18,18,10145135,1,1,0,0,1,0.0,0.0,IB,...,0.0,16.0,1.0,3.0,3.0,0.0,0.0,12.000000,52.0,1
41,41,10353432,1,4,0,0,,,,IV,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13.000000,49.0,1
43,43,10359877,1,3,1,1,0,1.0,0.0,IIIA,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,14.000000,53.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9900,9900,99343880,1,1,2,0,,,,IA,...,0.0,68.0,1.0,2.0,2.0,0.0,0.0,16.000000,47.0,1
9913,9913,99443402,1,3,0,1,,,,IIB,...,0.0,22.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0,1
9931,9931,99656792,1,1,0,0,,,,IA,...,1.0,49.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0,1
9933,9933,99690760,1,1,0,0,,,,IA,...,0.0,20.0,1.0,1.0,1.0,0.0,0.0,18.000000,49.0,1


Tras la realización, obtenemos que 1196 de los pacientes han presenciado una recaída , por ello mismo se le podría aplicar el tratamiento de los 40.000 € , que reduce así el rieso de más recaída, y el resto de los pacientes no serían de alto rieso a día de hoy
Ahora bien, cabe la posibilidad de que algunos de ellos, hayan muerto, por eso comprobamos la columna de death, si está nula, entonces a ellos se les aplicará el tratamiento mencionado, en otro caso no puesto que no han sobrevivido a la enfermedad

In [18]:
df_final[(df_final['recaida'] == 1) & (df_final['death_date'].isnull())]

,Unnamed: 0,ehr,n_tumor,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,...,her2,ki67,pr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age,recaida
1,1,10020495,1,1,0,0,,,,IA,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0,1
17,17,10144916,1,1,0,0,1,0.0,1.0,IA,...,0.0,20.0,1.0,0.0,0.0,0.0,0.0,14.000000,49.0,1
41,41,10353432,1,4,0,0,,,,IV,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13.000000,49.0,1
44,44,10364469,1,1,1,0,4,0.0,0.0,IA,...,0.0,19.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0,1
54,54,10464026,1,0,0,0,,,,IV,...,0.0,50.0,0.0,0.0,0.0,0.0,0.0,13.000000,46.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9900,9900,99343880,1,1,2,0,,,,IA,...,0.0,68.0,1.0,2.0,2.0,0.0,0.0,16.000000,47.0,1
9913,9913,99443402,1,3,0,1,,,,IIB,...,0.0,22.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0,1
9931,9931,99656792,1,1,0,0,,,,IA,...,1.0,49.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0,1
9933,9933,99690760,1,1,0,0,,,,IA,...,0.0,20.0,1.0,1.0,1.0,0.0,0.0,18.000000,49.0,1


De los 1196 que poseen recaídas, 1052 siguen sobreviviendo, por ello que la aplicación del tratamiento se realiza sobre ese nº de casos, ya que el resto falleció. Esto se lleva a cabo en eras de conseguir que el paciente sobreviva ya que es el objetivo del estudio.

Además, si aplicaríamos a todos los pacientes (NO FALLECIDOS), podríamos determinar cuál es el beneficio que obtendríamos en cuanto a costes... Es decir, calculamos los pacientes que tienen death a nulo, que serían los totales , * 40.000 que sería aplicar el tratamiento, pero solo aplicados a los que tienen recaída, la diferencia sería un beneficio.... si se HUBIESE APLICADO A TODOS


## **DESARROLLO DE MODELO PREDICTIVO DE RECAÍDAS**

Vamos a implementar un modelo que nos permita predecir si un paciente , dados sus datos iniciales ... es decir, los del primer tumor, va a tener o no una posible recaída.Para ello, tenemos 1196 pacientes que han poseido recaidas, y el resto NO. Para ello, incorporamos una variable objetivo, que es recaida, donde el valor 1 signifca tener recaida y un valor 0, signifca no poseer una recaída.

In [19]:
df_final

,Unnamed: 0,ehr,n_tumor,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,...,her2,ki67,pr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age,recaida
0,0,10011773,1,IS,0,0,,,,0,...,0.0,19.0,1.0,0.0,0.0,0.0,0.0,13.000000,49.0,0
1,1,10020495,1,1,0,0,,,,IA,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0,1
2,2,10030299,1,1,0,0,,,,IA,...,0.0,18.0,1.0,3.0,3.0,0.0,0.0,13.000000,49.0,0
3,3,10030824,1,2,1,0,2,2.0,0.0,IIIA,...,0.0,18.0,1.0,0.0,0.0,0.0,0.0,13.000000,44.0,0
4,4,10041592,1,1,1,0,4,0.0,0.0,IB,...,0.0,65.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9966,9966,99966287,1,1,1,0,1,0.0,0.0,IB,...,0.0,18.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0,0
9967,9967,99981192,1,1,0,0,,,,IA,...,0.0,14.0,0.0,3.0,3.0,0.0,0.0,11.000000,49.0,0
9968,9968,99988958,1,0,2,0,1,1.0,0.0,IIIA,...,0.0,67.0,1.0,0.0,0.0,0.0,0.0,13.110481,49.0,0
9969,9969,99992350,1,2,0,0,3,2.0,0.0,IIA,...,0.0,16.0,0.0,4.0,3.0,1.0,0.0,11.000000,55.0,0


In [20]:
df_final['recaida'].value_counts()

0    8775
1    1196
Name: recaida, dtype: int64

Antes de realizar el modelo, vemos que los datos están muy desbalanceados,por lo cuál es una consideración importante de tomar en el desarrollo del modelo.

Existen varias alternativas:
- Oversampling de la Clase Minoritaria: Aumenta la cantidad de instancias de la clase minoritaria para equilibrar las proporciones.
- Undersampling de la Clase Mayoritaria: Reduce la cantidad de instancias de la clase mayoritaria para equilibrar las proporciones.
- Ponderación de Clases: algunos algoritmos de clasificación permiten asignar pesos a las clases para dar más importancia a la clase minoritaria.

Primero de nada separamos las variables en X e Y, donde la Y es el objetivo, es decir, la recaída.

In [21]:
# Suponiendo que 'df_seleccionado' es tu DataFrame

# Asignar 'SIN TRATAMIENTO' a las columnas específicas basado en la condición
df_final.loc[df_final['stage_after_neo'] == 'SIN TRATAMIENTO', 
                    ['t_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj']] = 'SIN TRATAMIENTO'

df_final

,Unnamed: 0,ehr,n_tumor,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,...,her2,ki67,pr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age,recaida
0,0,10011773,1,IS,0,0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,0,...,0.0,19.0,1.0,0.0,0.0,0.0,0.0,13.000000,49.0,0
1,1,10020495,1,1,0,0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0,1
2,2,10030299,1,1,0,0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,...,0.0,18.0,1.0,3.0,3.0,0.0,0.0,13.000000,49.0,0
3,3,10030824,1,2,1,0,2,2.0,0.0,IIIA,...,0.0,18.0,1.0,0.0,0.0,0.0,0.0,13.000000,44.0,0
4,4,10041592,1,1,1,0,4,0.0,0.0,IB,...,0.0,65.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9966,9966,99966287,1,1,1,0,1,0.0,0.0,IB,...,0.0,18.0,0.0,0.0,0.0,0.0,0.0,13.110481,49.0,0
9967,9967,99981192,1,1,0,0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,...,0.0,14.0,0.0,3.0,3.0,0.0,0.0,11.000000,49.0,0
9968,9968,99988958,1,0,2,0,1,1.0,0.0,IIIA,...,0.0,67.0,1.0,0.0,0.0,0.0,0.0,13.110481,49.0,0
9969,9969,99992350,1,2,0,0,3,2.0,0.0,IIA,...,0.0,16.0,0.0,4.0,3.0,1.0,0.0,11.000000,55.0,0


In [22]:
columnas_seleccionadas = [
    'ehr','n_tumor', 't_category', 'n_category', 'm_category', 'stage_diagnosis',
    't_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj', 'stage_after_neo',
    'grade', 'ductal', 'lobular', 'neoadjuvant',
    'er', 'pr', 'her2', 'ki67','recaida'
]

df_modelo = df_final[columnas_seleccionadas]
df_modelo


,ehr,n_tumor,t_category,n_category,m_category,stage_diagnosis,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_after_neo,grade,ductal,lobular,neoadjuvant,er,pr,her2,ki67,recaida
0,10011773,1,IS,0,0,0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,1,0.0,0.0,0,0.0,1.0,0.0,19.0,0
1,10020495,1,1,0,0,IA,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,2,1.0,0.0,0,1.0,0.0,0.0,9.0,1
2,10030299,1,1,0,0,IA,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,1,1.0,0.0,0,1.0,1.0,0.0,18.0,0
3,10030824,1,2,1,0,IIIA,2,2.0,0.0,IIIA,3,0.0,1.0,1,1.0,1.0,0.0,18.0,0
4,10041592,1,1,1,0,IB,4,0.0,0.0,IA,2,1.0,0.0,1,0.0,0.0,0.0,65.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9966,99966287,1,1,1,0,IB,1,0.0,0.0,IIA,3,1.0,0.0,1,1.0,0.0,0.0,18.0,0
9967,99981192,1,1,0,0,IA,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,2,0.0,0.0,0,1.0,0.0,0.0,14.0,0
9968,99988958,1,0,2,0,IIIA,1,1.0,0.0,IIA,1,1.0,0.0,1,1.0,1.0,0.0,67.0,0
9969,99992350,1,2,0,0,IIA,3,2.0,0.0,IIIA,2,1.0,0.0,1,0.0,0.0,0.0,16.0,0


In [23]:
columnas_seleccionadas_2 = [
    't_category', 'n_category', 'm_category', 'stage_diagnosis',
    't_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj', 'stage_after_neo',
    'grade', 'ductal', 'lobular', 'neoadjuvant',
    'er', 'pr', 'her2', 'ki67'
]
X = df_modelo[columnas_seleccionadas_2]
y = df_modelo['recaida']

In [24]:
print('Original dataset shape %s' % Counter(y))
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))

Original dataset shape Counter({0: 8775, 1: 1196})
Resampled dataset shape Counter({0: 1196, 1: 1196})


In [25]:
X_res.dtypes

t_category                  object
n_category                   int64
m_category                   int64
stage_diagnosis             object
t_category_after_neoadj     object
n_category_after_neoadj     object
m_category_after_neoadj     object
stage_after_neo             object
grade                        int64
ductal                     float64
lobular                    float64
neoadjuvant                  int64
er                         float64
pr                         float64
her2                       float64
ki67                       float64
dtype: object

In [26]:
# Suponiendo que 'df_seleccionado' es tu DataFrame

# Lista de columnas que deseas convertir a tipo 'object' excepto 'ki67'
columnas_object = ['t_category', 'n_category', 'm_category', 'stage_diagnosis',
                    't_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj', 'stage_after_neo',
                    'grade', 'ductal', 'lobular', 'neoadjuvant', 'er', 'pr', 'her2','ki67']

# Convertir las columnas a tipo 'object' excepto 'ki67'
X_res[columnas_object] = X_res[columnas_object].astype(str)


In [27]:
X_res.dtypes

t_category                 object
n_category                 object
m_category                 object
stage_diagnosis            object
t_category_after_neoadj    object
n_category_after_neoadj    object
m_category_after_neoadj    object
stage_after_neo            object
grade                      object
ductal                     object
lobular                    object
neoadjuvant                object
er                         object
pr                         object
her2                       object
ki67                       object
dtype: object

In [28]:
for columna in X_res.columns:
    valores_unicos = X_res[columna].unique()
    print(f'Valores únicos en la columna {columna}:\n{valores_unicos}\n')


Valores únicos en la columna t_category:
['0' '1' '3' 'IS' '2' '4']

Valores únicos en la columna n_category:
['1' '0' '2' '3']

Valores únicos en la columna m_category:
['1' '0']

Valores únicos en la columna stage_diagnosis:
['IB' 'IA' 'IIIA' 'IV' '0' 'IIIC' 'IIA' 'IIB' 'IIIB']

Valores únicos en la columna t_category_after_neoadj:
['SIN TRATAMIENTO' '3' '0' 'IS' '2' '1' '4']

Valores únicos en la columna n_category_after_neoadj:
['SIN TRATAMIENTO' '0.0' '1.0' '3.0' '2.0']

Valores únicos en la columna m_category_after_neoadj:
['SIN TRATAMIENTO' '0.0' '1.0']

Valores únicos en la columna stage_after_neo:
['SIN TRATAMIENTO' 'IA' 'IB' '0' 'IIA' 'IIB' 'IIIA' 'IIIB' 'IV' 'IIIC']

Valores únicos en la columna grade:
['2' '1' '3']

Valores únicos en la columna ductal:
['1.0' '0.0']

Valores únicos en la columna lobular:
['0.0' '1.0']

Valores únicos en la columna neoadjuvant:
['0' '1']

Valores únicos en la columna er:
['1.0' '0.0']

Valores únicos en la columna pr:
['1.0' '0.0']

Valores 

In [29]:
# Suponiendo que 'columna' es el nombre de la columna que te interesa
columna = 'ductal'

# Itera sobre los valores de la columna e imprime el tipo de datos para cada valor
for valor in X_res[columna]:
    print(f'Tipo de datos para el valor {valor}: {type(valor)}')

Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos

In [30]:
X_res[columna] = X_res[columna].astype(str)

In [31]:
# Suponiendo que 'X_res' es tu DataFrame
label_encoder = LabelEncoder()

# Itera sobre todas las columnas y aplica el LabelEncoder
for columna in X_res.columns:
    X_res[columna] = label_encoder.fit_transform(X_res[columna])

# Ahora, todas las columnas en X_res contienen valores numéricos después de la codificación

In [32]:
X_res

,t_category,n_category,m_category,stage_diagnosis,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_after_neo,grade,ductal,lobular,neoadjuvant,er,pr,her2,ki67
9611,0,1,1,2,6,4,2,9,1,1,0,0,1,1,0,8
6742,0,1,0,1,3,0,0,1,0,1,0,1,0,1,0,10
3858,1,0,0,1,6,4,2,9,1,1,0,0,1,1,0,15
7397,0,2,0,5,0,1,0,2,0,1,0,1,1,1,0,19
1159,3,0,0,8,5,0,0,0,1,0,0,1,1,1,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9900,1,2,0,1,6,4,2,9,1,0,0,0,1,1,0,66
9913,3,0,1,4,6,4,2,9,2,0,1,0,1,0,0,16
9931,1,0,0,1,6,4,2,9,0,0,0,0,1,0,1,45
9933,1,0,0,1,6,4,2,9,0,1,0,0,1,1,0,14


In [33]:
y_res

9611    0
6742    0
3858    0
7397    0
1159    0
       ..
9900    1
9913    1
9931    1
9933    1
9949    1
Name: recaida, Length: 2392, dtype: int64

In [34]:
# Suponiendo que 'X_res' es tu DataFrame con las variables ya transformadas por LabelEncoder
# y 'y' es tu variable objetivo

# Inicializa el modelo de árbol de decisión
modelo_arbol = DecisionTreeClassifier()

# Realiza la validación cruzada y obtén las puntuaciones de precisión
scores = cross_val_score(modelo_arbol, X_res, y_res, cv=10, scoring='accuracy')

In [35]:
# Muestra las puntuaciones de precisión para cada pliegue
print(f'Puntuaciones de precisión por pliegue: {scores}')

# Muestra la precisión promedio
print(f'Precisión promedio: {scores.mean()}')

Puntuaciones de precisión por pliegue: [0.44583333 0.48333333 0.47698745 0.47280335 0.49790795 0.48535565
 0.46443515 0.50627615 0.43514644 0.51882845]
Precisión promedio: 0.4786907252440725


In [36]:
# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)
# Listamos los modelos a evaluar
modelos = [
    ('Random Forest', RandomForestClassifier()),
    ('SVM', SVC()),
    ('Logistic Regression', LogisticRegression()),
    ('KNN', KNeighborsClassifier()),
    ('Gradient Boosting', GradientBoostingClassifier()),
    ('Decision Tree', DecisionTreeClassifier())
]

# Diccionario para almacenar los resultados de la validación cruzada
resultados = {}

In [37]:
# Itera sobre cada modelo
for nombre, modelo in modelos:
    # Realiza la validación cruzada y obtén las puntuaciones de precisión
    scores = cross_val_score(modelo, X_train, y_train, cv=5, scoring='accuracy')
    
    # Almacena las puntuaciones en el diccionario de resultados
    resultados[nombre] = scores.mean()

# Encuentra el modelo con el mejor rendimiento
mejor_modelo = max(resultados, key=resultados.get)
print(f'Mejor modelo: {mejor_modelo} con precisión media: {resultados[mejor_modelo]}')

# Entrena el mejor modelo en el conjunto de entrenamiento completo
modelo_final = next(modelo for nombre, modelo in modelos if nombre == mejor_modelo)
modelo_final.fit(X_train, y_train)

# Evalúa el rendimiento en el conjunto de prueba
predicciones = modelo_final.predict(X_test)
precision_test = accuracy_score(y_test, predicciones)
print(f'Precisión en el conjunto de prueba: {precision_test}')

/Users/cnebreac/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/cnebreac/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no lo

Mejor modelo: Logistic Regression con precisión media: 0.4939920440720134
Precisión en el conjunto de prueba: 0.4780793319415449


In [38]:
resultados

{'Random Forest': 0.4908698207865706,
 'SVM': 0.4913687750331497,
 'Logistic Regression': 0.4939920440720134,
 'KNN': 0.4924322994272279,
 'Gradient Boosting': 0.4725383784670485,
 'Decision Tree': 0.4809126078219622}